# Demonstration

This notebook serves as a demonstration of the workflow of the entire project from querying data from the database to collecting predictions for the reviews from our trained model.

## Import modules

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

%run .engine.py
engine = create_engine(LOGIN)

## Query the data

In [ ]:
query = """
SELECT reviews.stars as stars, 
reviews.text as text, 
reviews.useful as useful, 
reviews.funny as funny, 
business.state as state,
business.categories as categories
FROM reviews
INNER JOIN business ON reviews.business_id = business.business_id
"""

df = pd.read_sql_query(query, engine)
df.shape'''